In [36]:
import torch
import torchvision
from torchvision.transforms import transforms
from torchvision.io import read_image
import matplotlib.pyplot as plt


In [37]:
training_data = torchvision.datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),

)

test_data = torchvision.datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)


Files already downloaded and verified
Files already downloaded and verified


In [38]:
labels_map = ['plane',
              'car',
              'bird',
              'cat',
              'deer',
              'dog',
              'frog',
              'horse',
              'ship',
              'truck']

figure = plt.figure(figsize=(8,8))
cols, rows = 4,4
for i in range(1, cols * rows +1):
    sample_idx = torch.randint(len(training_data), size=(1,)).item()
    img, label = training_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    plt.imshow(img.permute(1,2,0))
plt.show()


In [76]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset

class CustomDataset(Dataset):
  # 클래스 초기화
  def __init__(self, data, transforms=None):
    # 데이터 저장
    self.data = data
    # 데이터에 적용할 전처리 함수 저장
    self.transforms = transforms

  # 데이터셋의 길이
  def __len__(self):
    return len(self.data)

  # 데이터셋에서 특정 인덱스의 데이터 가져오기
  def __getitem__(self, index):
    sample = self.data[index]
    # 변환 함수가 지정되어 있다면 샘플에 적용
    if self.transforms:
      sample = self.transforms(sample)

    return sample

train_dataset = CustomDataset(training_data)
test_dataset = CustomDataset(test_data)

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([64, 3, 32, 32])
Labels batch shape: torch.Size([64])


In [ ]:
img = train_features[0]
label = train_labels[0]
plt.imshow(img.permute(1,2,0))
plt.show
print(f"Label: {label}")

Label: 4


In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cuda device


In [57]:
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),  # 입력 채널 3, 출력 채널 16, 커널 크기 3, 패딩 1
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 출력 크기: 16x16
            nn.Conv2d(16, 32, 3, padding=1),  # 입력 채널 16, 출력 채널 32, 커널 크기 3, 패딩 1
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 출력 크기: 8x8
            nn.Conv2d(32, 64, 3, padding=1),  # 입력 채널 32, 출력 채널 64, 커널 크기 3, 패딩 1
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 출력 크기: 4x4
            nn.Flatten(),  # 4x4x64 = 1024
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10)  # CIFAR10은 10개 클래스
        )

    def forward(self,x):
      logits = self.cnn_layers(x)
      return logits


In [69]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=1024, out_features=512, bias=True)
    (11): ReLU()
    (12): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [70]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [71]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred= model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [72]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X,y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred,y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \fake")

In [73]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n-------------------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306887  [    0/50000]
loss: 2.059781  [ 6400/50000]
loss: 1.662648  [12800/50000]
loss: 1.598102  [19200/50000]
loss: 1.409457  [25600/50000]
loss: 1.541087  [32000/50000]
loss: 1.449364  [38400/50000]
loss: 1.184576  [44800/50000]
Test Error: 
 Accuracy: 51.4%, Avg loss: 1.339438 ake
Epoch 2
-------------------------------
loss: 1.398550  [    0/50000]
loss: 1.288836  [ 6400/50000]
loss: 1.322022  [12800/50000]
loss: 1.368925  [19200/50000]
loss: 1.213043  [25600/50000]
loss: 1.298174  [32000/50000]
loss: 1.328359  [38400/50000]
loss: 1.132032  [44800/50000]
Test Error: 
 Accuracy: 58.6%, Avg loss: 1.174171 ake
Epoch 3
-------------------------------
loss: 0.917128  [    0/50000]
loss: 1.048774  [ 6400/50000]
loss: 1.220113  [12800/50000]
loss: 1.160512  [19200/50000]
loss: 1.226804  [25600/50000]
loss: 1.058453  [32000/50000]
loss: 1.179615  [38400/50000]
loss: 0.712501  [44800/50000]
Test Error: 
 Accuracy: 63.0%, Avg loss: 1.038654 